In [ ]:
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from data import load_train_data, load_test_data

In [ ]:
image_height, image_width = 96, 96
smoothness = 1.0
work_dir = 'train'

In [ ]:
from keras import backend as K

def dice_coef(y_true, y_pred, smooth=0.9):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [ ]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], image_height, image_width), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (image_width, image_height))
    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p


In [ ]:
from keras.layers import MaxPooling2D, Input, Conv2D, Activation, Conv2DTranspose, UpSampling2D, Dropout, Concatenate
from keras.models import Sequential
from keras.optimizers import Adam

unet = Sequential()

unet.add(Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', input_shape=(96, 96, 1)))
unet.add(Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(MaxPooling2D(pool_size=(2, 2)))

unet.add(Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
# unet.add(Activation('relu'))
unet.add(MaxPooling2D(pool_size=(2, 2)))

unet.add(Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(MaxPooling2D(pool_size=(2, 2)))

unet.add(Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Dropout(0.5))
unet.add(MaxPooling2D(pool_size=(2, 2)))

unet.add(Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Dropout(0.5))

unet.add(Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(UpSampling2D(2))
unet.add(Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))

unet.add(Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(UpSampling2D(2))
unet.add(Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
         
unet.add(Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(UpSampling2D(2))
unet.add(Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))

unet.add(Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(UpSampling2D(2))
unet.add(Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))


unet.add(Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'))
unet.add(Conv2D(1, 1, activation = 'sigmoid', padding = 'same', kernel_initializer = 'he_normal'))

unet.summary()

unet.compile(optimizer=Adam(),
              loss=dice_coef_loss,
              metrics=[dice_coef])

In [ ]:
import os
import numpy as np

from skimage.io import imsave, imread

x_train, y_train_mask = load_train_data()

x_train = preprocess(x_train)
y_train_mask = preprocess(y_train_mask)

x_train = x_train.astype('float32')
mean = np.mean(x_train)
std = np.std(x_train)

x_train -= mean
x_train /= std

y_train_mask = y_train_mask.astype('float32')
y_train_mask /= 255.

unet.fit(x_train, y_train_mask, batch_size=32, epochs=20, verbose=1, shuffle=True, validation_split=0.2)

x_test, y_test_mask = load_test_data()
x_test = preprocess(x_test)

x_test = x_test.astype('float32')
x_test -= mean
x_test /= std

y_test_pred = unet.predict(x_test, verbose=1)

for image, image_id in zip(y_test_pred, y_test_mask):
    image = (image[:, :, 0] * 255.).astype(np.uint8)
    imsave(os.path.join(work_dir, str(image_id) + '.png'), image)